In [2]:
from rdkit import rdBase, Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import IPythonConsole
import sys, py3Dmol
import pandas as pd
from mordred import Calculator,descriptors
import numpy as np
import math
# データ可視化ライブラリ
import matplotlib.pyplot as plt

In [4]:
df = pd.DataFrame(pd.read_csv('dataTg_new.csv'))
df = df[0:3]
#SMILESキーの入った列を指定
smiles = df["SMILES"]
df.head()

,Unnamed: 0,Tg,SMILES,NAME
0,1,279,C=CC(=O)OCc1ccccc1,Poly(benzyl acrylate)
1,2,383,C=CC(=O)Oc2ccc(c1ccccc1)cc2,Poly(4-biphenyl acrylate)
2,3,219,CCCCOC(=O)C=C,Poly(butyl acrylate)


In [5]:
mols = [Chem.MolFromSmiles(smile) for smile in smiles]

In [6]:
mols_from_sm = mols

#3次元構造発生用の関数を定義（力場が複数あります）
def DGuff(mols):
    DG_uff = []
    for mol in mols:
        mh = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mh, useBasicKnowledge=False, useExpTorsionAnglePrefs=False)
        if AllChem.UFFHasAllMoleculeParams(mh):
            AllChem.UFFOptimizeMolecule(mh)
            DG_uff.append(mh)
    return DG_uff

def DGmmff(mols):
    DG_mmff = []
    for mol in mols:
        mh = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mh, useBasicKnowledge=False, useExpTorsionAnglePrefs=False)
        if AllChem.MMFFHasAllMoleculeParams(mh):
            AllChem.MMFFOptimizeMolecule(mh)
            DG_mmff.append(mh)
    return DG_mmff

def ETDG(mols):
    ETDG_mols = []
    for mol in mols:
        mh = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mh, AllChem.ETDG())
        ETDG_mols.append(mh)
    return ETDG_mols

def KDG(mols):
    KDG_mols = []
    for mol in mols:
        mh = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mh, AllChem.KDG())
        KDG_mols.append(mh)
    return KDG_mols


def ETKDG(mols, version=1):
    ETKDG_mols = []
    for mol in mols:
        mh = Chem.AddHs(mol)
        if version == 1:
            p = AllChem.ETKDG()
        elif version == 2:
            p = AllChem.ETKDGv2()
        else:
            print('invalid input')
        AllChem.EmbedMolecule(mh, p)
        ETKDG_mols.append(mh)
    return ETKDG_mols
 
DG_m = DGuff(mols_from_sm)
ETDG_m = ETDG(mols_from_sm)
ETKDGv1_m = ETKDG(mols_from_sm)
ETKDGv2_m = ETKDG(mols_from_sm, 2)

In [10]:
#構造発生回数の設定（分子力学による3次元構造は、分布を持つため）
iteration_num = 10
set_num = list(range(iteration_num))
rows_num =list(range(len(DG_m)))

In [11]:
#3D記述子のみを計算する関数を設定
calc_dummy = Calculator(descriptors, ignore_3D=False)
my_desc_names = ["PNSA1","PNSA2","PNSA3","PNSA4","PNSA5","PPSA1","PPSA2","PPSA3","PPSA4","PPSA5","DPSA1","DPSA2","DPSA3","DPSA4","DPSA5","FNSA1","FNSA2","FNSA3","FNSA4","FNSA5","FPSA1","FPSA2","FPSA3","FPSA4","FPSA5","WNSA1","WNSA2","WNSA3","WNSA4","WNSA5","WPSA1","WPSA2","WPSA3","WPSA4","WPSA5","RNCS","RPCS","TASA","TPSA","RASA","RPSA","GeomDiameter","GeomRadius","GeomShapeIndex","GeomPetitjeanIndex","GRAV","GRAVH","GRAVp","GRAVHp","Mor01","Mor02","Mor03","Mor04","Mor05","Mor06","Mor07","Mor08","Mor09","Mor10","Mor11","Mor12","Mor13","Mor14","Mor15","Mor16","Mor17","Mor18","Mor19","Mor20","Mor21","Mor22","Mor23","Mor24","Mor25","Mor26","Mor27","Mor28","Mor29","Mor30","Mor31","Mor32","Mor01m","Mor02m","Mor03m","Mor04m","Mor05m","Mor06m","Mor07m","Mor08m","Mor09m","Mor10m","Mor11m","Mor12m","Mor13m","Mor14m","Mor15m","Mor16m","Mor17m","Mor18m","Mor19m","Mor20m","Mor21m","Mor22m","Mor23m","Mor24m","Mor25m","Mor26m","Mor27m","Mor28m","Mor29m","Mor30m","Mor31m","Mor32m","Mor01v","Mor02v","Mor03v","Mor04v","Mor05v","Mor06v","Mor07v","Mor08v","Mor09v","Mor10v","Mor11v","Mor12v","Mor13v","Mor14v","Mor15v","Mor16v","Mor17v","Mor18v","Mor19v","Mor20v","Mor21v","Mor22v","Mor23v","Mor24v","Mor25v","Mor26v","Mor27v","Mor28v","Mor29v","Mor30v","Mor31v","Mor32v","Mor01se","Mor02se","Mor03se","Mor04se","Mor05se","Mor06se","Mor07se","Mor08se","Mor09se","Mor10se","Mor11se","Mor12se","Mor13se","Mor14se","Mor15se","Mor16se","Mor17se","Mor18se","Mor19se","Mor20se","Mor21se","Mor22se","Mor23se","Mor24se","Mor25se","Mor26se","Mor27se","Mor28se","Mor29se","Mor30se","Mor31se","Mor32se","Mor01p","Mor02p","Mor03p","Mor04p","Mor05p","Mor06p","Mor07p","Mor08p","Mor09p","Mor10p","Mor11p","Mor12p","Mor13p","Mor14p","Mor15p","Mor16p","Mor17p","Mor18p","Mor19p","Mor20p","Mor21p","Mor22p","Mor23p","Mor24p","Mor25p","Mor26p","Mor27p","Mor28p","Mor29p","Mor30p","Mor31p","Mor32p","MOMI-X","MOMI-Y","MOMI-Z","PBF"]
my_descs = []
for i, desc in enumerate(calc_dummy.descriptors):
    if desc.__str__()  in my_desc_names:
       my_descs.append(desc)

calc_real = Calculator(my_descs, ignore_3D=False)

ValueError: 0 is not descriptor

In [12]:
#各力場により、指定された回数だけ構造を発生、記述子計算
descriptors = {}

for i in set_num:
    descriptors[i] = pd.DataFrame(calc_real.pandas(ETKDG(mols_from_sm, 2)))
    descriptors[i] = pd.concat([df, descriptors[i]], axis = 1)
    detkdg = pd.DataFrame(calc_real.pandas(ETKDG(mols_from_sm)))
    descriptors[i] = pd.concat([descriptors[i], detkdg.add_suffix("_ETKDG")], axis = 1)
    detdg = pd.DataFrame(calc_real.pandas(ETDG(mols_from_sm)))
    descriptors[i] = pd.concat([descriptors[i], detdg.add_suffix("_ETDG")], axis = 1)
    dkdg = pd.DataFrame(calc_real.pandas(KDG(mols_from_sm)))
    descriptors[i] = pd.concat([descriptors[i], dkdg.add_suffix("_KDG")], axis = 1)
    ddguff = pd.DataFrame(calc_real.pandas(DGuff(mols_from_sm)))
    descriptors[i] = pd.concat([descriptors[i], ddguff.add_suffix("_DGuff")], axis = 1)
    ddgmmff = pd.DataFrame(calc_real.pandas(DGmmff(mols_from_sm)))
    descriptors[i] = pd.concat([descriptors[i], ddgmmff.add_suffix("_DGmmff")], axis = 1)


100%|██████████| 3/3 [00:00<00:00, 57.39it/s]


In [13]:
descriptors[4]#n回目の構造発生＋記述子計算結果

,Unnamed: 0,Tg,SMILES,NAME,PNSA1,PNSA2,PNSA3,PNSA4,PNSA5,PPSA1,...,Mor27p_DGmmff,Mor28p_DGmmff,Mor29p_DGmmff,Mor30p_DGmmff,Mor31p_DGmmff,Mor32p_DGmmff,MOMI-X_DGmmff,MOMI-Y_DGmmff,MOMI-Z_DGmmff,PBF_DGmmff
0,1,279,C=CC(=O)OCc1ccccc1,Poly(benzyl acrylate),158.535765,-175.076316,-20.192088,-7.958014,-19.452924,213.366919,...,-0.153976,-0.070752,0.044527,-0.088776,0.251326,-0.102912,1050.062926,974.176097,264.808014,0.858225
1,2,383,C=CC(=O)Oc2ccc(c1ccccc1)cc2,Poly(4-biphenyl acrylate),209.339044,-257.693039,-21.858047,-8.885967,-18.406646,239.580117,...,-0.310074,0.040439,0.153982,-0.048420,0.107388,-0.258429,2952.517302,2826.972448,259.882720,0.599983
2,3,219,CCCCOC(=O)C=C,Poly(butyl acrylate),103.106927,-96.756485,-18.502845,-4.607452,-16.126081,238.486927,...,-0.010023,-0.083746,-0.216880,-0.029566,0.322248,0.062995,772.990675,701.950059,122.693883,0.578317


In [14]:
#出来上がった記述子の統計値をとります
distribution = {}

for m in rows_num:

    numbers = pd.DataFrame()
    for n in set_num:
        number = pd.Series(descriptors[n].iloc[m])
        numbers = numbers.append(number, ignore_index=False)
    distribution[m] = numbers


In [15]:
distribution[1]

,DPSA1,DPSA1_DGmmff,DPSA1_DGuff,DPSA1_ETDG,DPSA1_ETKDG,DPSA1_KDG,DPSA2,DPSA2_DGmmff,DPSA2_DGuff,DPSA2_ETDG,...,WPSA4_DGuff,WPSA4_ETDG,WPSA4_ETKDG,WPSA4_KDG,WPSA5,WPSA5_DGmmff,WPSA5_DGuff,WPSA5_ETDG,WPSA5_ETKDG,WPSA5_KDG
1,32.181737,53.001859,37.258761,51.107246,35.611275,33.529972,552.416872,560.953611,559.421638,546.913399,...,4.742637,4.671365,4.636504,4.626191,8.856018,9.511849,9.169098,9.031305,8.963909,8.943969
1,32.668591,52.369964,39.581755,59.707418,36.751147,37.175581,553.935350,563.852097,558.915343,561.005230,...,4.756767,4.985640,4.608911,4.615344,8.912127,9.593376,9.196415,9.638904,8.910560,8.922998
1,32.764231,51.011463,9.182283,31.963449,36.086199,25.142283,556.153402,562.537045,554.233627,553.605611,...,4.390089,4.597690,4.671659,4.537234,8.982990,9.525497,8.487506,8.888867,9.031874,8.771986
1,31.552602,50.064038,37.537976,47.968873,35.936317,31.789888,552.133067,562.836174,561.672375,561.149996,...,4.782128,4.874494,4.608911,4.634102,8.835646,9.517346,9.245448,9.424021,8.910562,8.959263
1,30.241073,54.409857,41.252880,67.018085,33.892628,59.608882,552.612358,559.761743,555.988657,553.156226,...,4.725087,4.924798,4.669073,4.947923,8.826329,9.499845,9.135168,9.521276,9.026874,9.565985
1,32.821255,50.041045,38.527481,61.389484,41.295050,38.160305,555.792830,560.429866,560.181425,549.079075,...,4.767293,4.803858,4.637196,4.669719,8.972795,9.439709,9.216766,9.287459,8.965246,9.028124
1,33.658530,52.608357,39.392668,54.567815,35.320853,25.686148,552.988911,562.022450,558.610442,561.945022,...,4.749965,4.951593,4.626200,4.588247,8.900976,9.538868,9.183266,9.573079,8.943987,8.870611
1,35.617513,54.424412,16.565949,57.395896,36.582239,59.316116,550.895832,560.366095,547.740593,561.371703,...,4.358575,4.969405,4.588491,5.046511,8.872035,9.519544,8.426578,9.607517,8.871083,9.756589
1,28.629185,54.166696,40.968796,45.561095,37.826722,45.525122,553.369093,559.112594,555.485871,550.602279,...,4.714181,4.678670,4.609678,4.832469,8.820022,9.474468,9.114084,9.045429,8.912043,9.342774
1,33.553843,51.855154,35.920399,46.041097,31.401018,57.543392,554.515618,560.370254,561.063027,555.653717,...,4.756504,4.765506,4.585719,4.971442,8.946539,9.471686,9.195908,9.213312,8.865723,9.611455


In [16]:
dfaugument = pd.DataFrame()

for l in rows_num:
    aug = distribution[l]
    dfaugument = dfaugument.append(aug, ignore_index = False)

In [17]:
#NAを入力してからcsvファイルを出力
df_summary = dfaugument.astype(str)
masks_3d = df_summary.apply(lambda d: d.str.contains('[a-zA-Z]' ,na=True))
df_summary = df_summary[~masks_3d]
df_summary = df_summary.astype(float)
df_summary = df_summary.fillna("NA")


In [18]:
df_summary

,DPSA1,DPSA1_DGmmff,DPSA1_DGuff,DPSA1_ETDG,DPSA1_ETKDG,DPSA1_KDG,DPSA2,DPSA2_DGmmff,DPSA2_DGuff,DPSA2_ETDG,...,WPSA4_DGuff,WPSA4_ETDG,WPSA4_ETKDG,WPSA4_KDG,WPSA5,WPSA5_DGmmff,WPSA5_DGuff,WPSA5_ETDG,WPSA5_ETKDG,WPSA5_KDG
0,60.037749,65.869404,79.163580,60.876532,81.176795,60.924559,409.980429,401.684200,402.471971,406.352969,...,4.057755,3.960453,3.955795,4.027075,6.800704,6.637121,6.866969,6.702305,6.694422,6.815049
0,59.710654,61.902463,60.187786,74.160633,78.400091,59.676762,408.272845,408.202187,411.167355,395.714959,...,4.041678,3.889608,3.955696,3.951286,6.742962,6.775202,6.839763,6.582413,6.694255,6.686792
0,63.367274,61.554951,63.194006,64.282592,42.071541,91.752191,409.456140,408.734830,411.862612,407.851622,...,4.082545,4.019210,3.662799,4.018498,6.836965,6.786162,6.908922,6.801740,6.198584,6.800534
0,62.583285,67.207772,63.385476,83.882075,60.109821,56.720717,409.145870,401.662407,411.865354,400.408584,...,4.084388,4.062887,4.001021,3.995756,6.815026,6.657132,6.912041,6.875655,6.770959,6.762049
0,54.831154,68.459728,62.128715,67.109344,81.765365,68.460925,410.704498,398.998771,414.899909,405.990102,...,4.128541,4.011395,3.958511,4.139196,6.740827,6.595183,6.986762,6.788514,6.699019,7.004793
0,82.158040,61.817116,59.999777,58.639320,79.204970,51.680038,400.111916,408.716400,411.117032,411.133628,...,4.039001,4.026592,3.978913,3.758422,6.839896,6.789715,6.835232,6.814233,6.733545,6.360407
0,81.367478,63.420053,76.094149,59.513060,57.635671,61.893844,397.273826,406.405839,400.563670,409.971371,...,3.994840,4.013544,3.910582,3.976064,6.740031,6.743683,6.760499,6.792152,6.617907,6.728724
0,78.634498,62.611159,58.542327,68.615010,63.143502,60.163435,399.590690,408.568187,410.941963,408.343990,...,4.022188,4.068407,4.018648,4.030590,6.769579,6.797622,6.806780,6.884997,6.800790,6.820998
0,81.944847,63.270928,36.527172,83.821887,62.359132,90.700038,398.470041,408.165019,411.699159,399.674818,...,3.830024,4.048860,4.039321,4.140060,6.785775,6.795540,6.481579,6.851916,6.835775,7.006256
0,69.131535,68.542312,78.234014,67.960711,59.825627,61.336116,411.958747,399.779256,400.723627,409.053415,...,4.017243,4.075365,3.961608,4.037478,7.005992,6.620220,6.798412,6.896771,6.704260,6.832656


In [27]:
df_summary_add = pd.concat([dfaugument["Tg"],dfaugument["SMILES"], dfaugument["NAME"], df_summary.drop(['Tg',"SMILES", "NAME"], axis = 1)], axis = 1)
df_summary_add
#df_summary_add.to_csv('DataTg_new_3d_augument.csv', index = False)

,Tg,SMILES,NAME,DPSA1,DPSA1_DGmmff,DPSA1_DGuff,DPSA1_ETDG,DPSA1_ETKDG,DPSA1_KDG,DPSA2,...,WPSA4_DGuff,WPSA4_ETDG,WPSA4_ETKDG,WPSA4_KDG,WPSA5,WPSA5_DGmmff,WPSA5_DGuff,WPSA5_ETDG,WPSA5_ETKDG,WPSA5_KDG
0,279.0,C=CC(=O)OCc1ccccc1,Poly(benzyl acrylate),60.037749,65.869404,79.163580,60.876532,81.176795,60.924559,409.980429,...,4.057755,3.960453,3.955795,4.027075,6.800704,6.637121,6.866969,6.702305,6.694422,6.815049
0,279.0,C=CC(=O)OCc1ccccc1,Poly(benzyl acrylate),59.710654,61.902463,60.187786,74.160633,78.400091,59.676762,408.272845,...,4.041678,3.889608,3.955696,3.951286,6.742962,6.775202,6.839763,6.582413,6.694255,6.686792
0,279.0,C=CC(=O)OCc1ccccc1,Poly(benzyl acrylate),63.367274,61.554951,63.194006,64.282592,42.071541,91.752191,409.456140,...,4.082545,4.019210,3.662799,4.018498,6.836965,6.786162,6.908922,6.801740,6.198584,6.800534
0,279.0,C=CC(=O)OCc1ccccc1,Poly(benzyl acrylate),62.583285,67.207772,63.385476,83.882075,60.109821,56.720717,409.145870,...,4.084388,4.062887,4.001021,3.995756,6.815026,6.657132,6.912041,6.875655,6.770959,6.762049
0,279.0,C=CC(=O)OCc1ccccc1,Poly(benzyl acrylate),54.831154,68.459728,62.128715,67.109344,81.765365,68.460925,410.704498,...,4.128541,4.011395,3.958511,4.139196,6.740827,6.595183,6.986762,6.788514,6.699019,7.004793
0,279.0,C=CC(=O)OCc1ccccc1,Poly(benzyl acrylate),82.158040,61.817116,59.999777,58.639320,79.204970,51.680038,400.111916,...,4.039001,4.026592,3.978913,3.758422,6.839896,6.789715,6.835232,6.814233,6.733545,6.360407
0,279.0,C=CC(=O)OCc1ccccc1,Poly(benzyl acrylate),81.367478,63.420053,76.094149,59.513060,57.635671,61.893844,397.273826,...,3.994840,4.013544,3.910582,3.976064,6.740031,6.743683,6.760499,6.792152,6.617907,6.728724
0,279.0,C=CC(=O)OCc1ccccc1,Poly(benzyl acrylate),78.634498,62.611159,58.542327,68.615010,63.143502,60.163435,399.590690,...,4.022188,4.068407,4.018648,4.030590,6.769579,6.797622,6.806780,6.884997,6.800790,6.820998
0,279.0,C=CC(=O)OCc1ccccc1,Poly(benzyl acrylate),81.944847,63.270928,36.527172,83.821887,62.359132,90.700038,398.470041,...,3.830024,4.048860,4.039321,4.140060,6.785775,6.795540,6.481579,6.851916,6.835775,7.006256
0,279.0,C=CC(=O)OCc1ccccc1,Poly(benzyl acrylate),69.131535,68.542312,78.234014,67.960711,59.825627,61.336116,411.958747,...,4.017243,4.075365,3.961608,4.037478,7.005992,6.620220,6.798412,6.896771,6.704260,6.832656
